In [ ]:
#Initialize Earth Engine and authorize access
import ee
ee.Authenticate()
ee.Initialize()
from ee import batch

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=WGhvUJklq2dQAzAdUqUo3Gtc1nn19jR8EspMzS2gXbw&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgeWYb-1Epde0x4tUmlIvBwN1lqrhF2HVvN-4Y1IH-bePQfVvWWgOw

Successfully saved authorization token.


In [ ]:
#LANDSAT 8 IMAGE COLLECTION
geometry = ee.Geometry.Polygon([[[7.696524,47.53076],[7.682018,47.53088],[7.668114,47.53325],[7.613525,47.55712],[7.524519,47.65209],[7.514477,47.67238],[7.506494,47.70237],[7.561067,48.08531],[7.574915,48.1258],[7.799582,48.5878],[7.834088,48.64439],[8.417351,49.58993],[9.308853,49.77173],[9.426613,49.79456],[9.654235,49.7939],[9.802208,49.73424],[10.093,49.54504],[10.12699,49.50871],[10.45383,48.9419],[10.51495,48.69663],[10.13231,47.67001],[10.07635,47.63602],[9.566016,47.53274],[9.511991,47.53246],[7.696524,47.53076]]], proj='EPSG:4326')

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2016-11-01', '2019-10-31').filterBounds(geometry)
#L8 2013-04-11 to 2019-10-31
count = l8.size()
print('Count: ', str(count.getInfo())+'\n')

Count:  569



In [ ]:
#LANDSAT 7 IMAGE COLLECTION
geometry = ee.Geometry.Polygon([[[7.696524,47.53076],[7.682018,47.53088],[7.668114,47.53325],[7.613525,47.55712],[7.524519,47.65209],[7.514477,47.67238],[7.506494,47.70237],[7.561067,48.08531],[7.574915,48.1258],[7.799582,48.5878],[7.834088,48.64439],[8.417351,49.58993],[9.308853,49.77173],[9.426613,49.79456],[9.654235,49.7939],[9.802208,49.73424],[10.093,49.54504],[10.12699,49.50871],[10.45383,48.9419],[10.51495,48.69663],[10.13231,47.67001],[10.07635,47.63602],[9.566016,47.53274],[9.511991,47.53246],[7.696524,47.53076]]], proj='EPSG:4326')

l8 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterDate('2007-01-01', '2013-04-10').filterBounds(geometry)
#L7 '2000-01-01', '2013-04-10'
count = l8.size()
print('Count: ', str(count.getInfo())+'\n')

Count:  581



In [ ]:
#Mask clouds and cloud shadows
def maskL8sr(image):
    #Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    #Get the pixel QA band.
    qa = image.select('pixel_qa')
    #Both flags should be set to zero, indicating clear conditions.
    shadow = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
    cloud = qa.bitwiseAnd(cloudsBitMask).eq(0)
    mask = shadow.And(cloud)
    return image.updateMask(mask)
l8_masked = l8.filterBounds(geometry).map(maskL8sr)


In [ ]:
#Clip Landsat images to study area
def boundary(image):
    return image.clip(geometry)

l8_masked_clip = l8_masked.map(boundary)

In [ ]:
#NDVI CALCULATION LANDSAT 8
def calc_ndvi(image):
    nir = image.select('B5')
    red = image.select('B4')
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    ndvi = ndvi.multiply(10000)
    return ndvi

ndvi_img = l8_masked_clip.map(calc_ndvi)

In [ ]:
#NDVI CALCULATION LANDSAT 7
def calc_ndvi(image):
    nir = image.select('B4')
    red = image.select('B3')
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    ndvi = ndvi.multiply(10000)
    return ndvi

ndvi_img = l8_masked_clip.map(calc_ndvi)

In [ ]:
#NDMI CALCULATION LANDSAT 8
def calc_ndmi(image):
    nir = image.select('B5')
    swir = image.select('B6')
    ndmi = nir.subtract(swir).divide(nir.add(swir)).rename('NDMI')
    ndmi = ndmi.multiply(10000)
    return ndmi

ndvi_img = l8_masked_clip.map(calc_ndmi)

In [ ]:
#NDMI CALCULATION LANDSAT 7
def calc_ndmi(image):
    nir = image.select('B4')
    swir = image.select('B5')
    ndmi = nir.subtract(swir).divide(nir.add(swir)).rename('NDMI')
    ndmi = ndmi.multiply(10000)
    return ndmi

ndvi_img = l8_masked_clip.map(calc_ndmi)

In [ ]:
#NDBI CALCULATION LANDSAT 8
def calc_ndbi(image):
    nir = image.select('B5')
    swir = image.select('B6')
    ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')
    ndbi=ndbi.multiply(10000)
    return ndbi

ndvi_img = l8_masked_clip.map(calc_ndbi)

In [ ]:
#NDBI CALCULATION LANDSAT 7
def calc_ndbi(image):
    nir = image.select('B4')
    swir = image.select('B5')
    ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')
    ndbi=ndbi.multiply(10000)
    return ndbi

ndvi_img = l8_masked_clip.map(calc_ndbi)

In [ ]:
#NDWI CALCULATION LANDSAT 8
def calc_ndwi(image):
    nir = image.select('B5')
    g = image.select('B3')
    ndwi = g.subtract(nir).divide(g.add(nir)).rename('NDWI')
    return ndwi

ndvi_img = l8_masked_clip.map(calc_ndwi)

In [ ]:
#NDWI CALCULATION LANDSAT 7
def calc_ndwi(image):
    nir = image.select('B4')
    g = image.select('B2')
    ndwi = g.subtract(nir).divide(g.add(nir)).rename('NDWI')
    return ndwi

ndvi_img = l8_masked_clip.map(calc_ndwi)

In [ ]:
#SAVI CALCULATION LANDSAT 8
def calc_savi(image):
    savi = image.expression('(1 + L) * ((NIR - RED) / (NIR + RED + L))', {
        'NIR': image.select('B5'),
        'RED': image.select('B4'),
        'L': 0.5
    }).float().rename('SAVI')
    savi=savi.multiply(10000)
    return savi

ndvi_img = l8_masked_clip.map(calc_savi)

In [ ]:
#SAVI CALCULATION LANDSAT 7
def calc_savi(image):
    #nir = image.select('B4')
    #red = image.select('B3')
    savi = image.expression('(1 + L) * ((NIR - RED) / (NIR + RED + L))', {
        'NIR': image.select('B4'),
        'RED': image.select('B3'),
        'L': 0.5
    }).float().rename('SAVI')
    savi=savi.multiply(10000)
    return savi

ndvi_img = l8_masked_clip.map(calc_savi)

In [ ]:
# Export images to Google Drive
# This is OK for small collections
collectionList = ndvi_img.toList(ndvi_img.size())
collectionSize = collectionList.size().getInfo()
for i in range(collectionSize):
    image = ee.Image(collectionList.get(i))
    name = image.get('system:index').getInfo()
    #Export NDMI
    task = ee.batch.Export.image.toDrive(
        image = image.toInt(),
        description = name,
        folder = 'ndvi_neu',
        scale = 30)
    task.start()
    print(task.status())

{'state': 'READY', 'description': 'LE07_193026_20070311', 'creation_timestamp_ms': 1639563180272, 'update_timestamp_ms': 1639563180272, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'HYZ7N4YTOKIJ2ZFAL56BPSRI', 'name': 'projects/earthengine-legacy/operations/HYZ7N4YTOKIJ2ZFAL56BPSRI'}
{'state': 'READY', 'description': 'LE07_193026_20070327', 'creation_timestamp_ms': 1639563184362, 'update_timestamp_ms': 1639563184362, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'GI6BZIPD3536NPCB3M5EGQJV', 'name': 'projects/earthengine-legacy/operations/GI6BZIPD3536NPCB3M5EGQJV'}
{'state': 'READY', 'description': 'LE07_193026_20070412', 'creation_timestamp_ms': 1639563190845, 'update_timestamp_ms': 1639563190845, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'TDBJBZD3UCYMFIMATSXCNZTR', 'name': 'projects/earthengine-legacy/operations/TDBJBZD3UCYMFIMATSXCNZTR'}
{'state': 'READY', 'description': 'LE07_193026_20070428', 'creation_timestamp_ms': 1639563195609, 'upda